# 지난 내용 복습

저번 시간 내용은 전부 그래프 그리는 내용이라서,  
그냥 7주차 내용 복습

In [38]:
import numpy as np
from typing import Tuple, List
import weakref

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise ValueError(f"{type(data)}는 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, f):
        self.creator = f
        self.generation = f.generation + 1

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        seen_set = set()
        funcs = []

        def add_func(f):
            if f not in seen_set:
                seen_set.add(f)
                funcs.append(f)
            funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            xs, gys = f.inputs, [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,

            for x, gx in zip(xs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)

class Function:
    def __call__(self, *inputs:List[Variable]):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,
        
        self.generation = 0
        for input in inputs:
            self.generation = max(self.generation, input.generation)
        outputs = [Variable(as_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]

        return output
    
    def forward(self, x:np.array):
        return NotImplementedError()
    
    def backward(self, gy:np.array):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x:np.array) -> np.array:
        return np.exp(x)
    
    def backward(self, gy:np.array) -> np.array:
        return gy * np.exp(self.inputs[0].data)
    
class Square(Function):
    def forward(self, x:np.array) -> np.array:
        return x ** 2
    
    def backward(self, gy:np.array) -> np.array:
        return gy * 2 * self.inputs[0].data
    
class Add(Function):
    def forward(self, x1:np.array, x0:np.array) -> np.array:
        return x0 + x1
    
    def backward(self, gy:np.array) -> Tuple[np.array, np.array]:
        return gy, gy
    
def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)


In [39]:
x = Variable(np.array(2.00))
a = square(x)
y = add(square(a), square(a))
y.backward()
print(y.data)
print(x.grad)

32.0
64.0


# 이번주에는 총 2가지를 배운다 (step19~22)
1. 변수 사용성 개선
2. 메서드 오버로딩 1 & 2

In [1]:
import numpy as np
from dezero import Variable

In [2]:
def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

In [3]:
x = Variable(np.array(2.0))
y = f(x)
y.backward(create_graph=True)
print(x.grad)
gx = x.grad
gx.backward()
print(x.grad)

variable(24.0)
variable(68.0)


In [4]:
x = Variable(np.array(2.0))
iters = 10

for i in range(iters):
    print(i,x)
    y = f(x)
    x.cleargrad()
    y.backward(create_graph=True)
    gx = x.grad

    x.cleargrad()
    gx.backward()
    gx2 = x.grad
    #print(gx, gx2)

    x.data -= gx.data / gx2.data

0 variable(2.0)
1 variable(1.4545454545454546)
2 variable(1.1510467893775467)
3 variable(1.0253259289766978)
4 variable(1.0009084519430513)
5 variable(1.0000012353089454)
6 variable(1.000000000002289)
7 variable(1.0)
8 variable(1.0)
9 variable(1.0)


In [5]:
import dezero.functions as F

x = Variable(np.array(1.0))
y = F.sin(x)
y.backward(create_graph=True)

for i in range(3):
    gx = x.grad
    x.cleargrad()
    gx.backward(create_graph=True)
    print(x.grad)

variable(-0.8414709848078965)
variable(-0.5403023058681398)
variable(0.8414709848078965)
